In [1]:
import pysam

In [2]:
header = pysam.VariantHeader()

In [3]:
header.add_meta('fileformat', value='VCFv4.1')

In [4]:
header.formats.add('GT', 1, 'String', 'Genotype')
header.formats.add('AD', 1, 'Integer', 'Allelic depths for the ref and alt alleles in the order listed')

In [5]:
header.add_sample('SAMPLE01')
header.add_sample('SAMPLE02')
header.add_sample('SAMPLE03')

In [6]:
chromosomes = (['chr' + str(num) for num in range(1, 11)] + ['X', 'Y'])
chromosomes = sorted(chromosomes, reverse=True)
# print(chromosomes)
for chromosome in chromosomes:
    header.contigs.add(chromosome)

In [7]:
header.info.add('END', 1, 'Integer', 'Stop position of the interval')
print(list(header.contigs))
print str(header.info.header)

['chr9', 'chr8', 'chr7', 'chr6', 'chr5', 'chr4', 'chr3', 'chr2', 'chr10', 'chr1', 'Y', 'X']
##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##FORMAT=<ID=GT,Number=1,Type=String,Description=Genotype>
##FORMAT=<ID=AD,Number=1,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##contig=<ID=chr9>
##contig=<ID=chr8>
##contig=<ID=chr7>
##contig=<ID=chr6>
##contig=<ID=chr5>
##contig=<ID=chr4>
##contig=<ID=chr3>
##contig=<ID=chr2>
##contig=<ID=chr10>
##contig=<ID=chr1>
##contig=<ID=Y>
##contig=<ID=X>
##INFO=<ID=END,Number=1,Type=Integer,Description="Stop position of the interval">
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	SAMPLE01	SAMPLE02	SAMPLE03



In [8]:
# vcf_in = pysam.VariantFile('./test.vcf', 'r')
# header = vcf_in.header
vcf = pysam.VariantFile('./output.vcf', 'w', header=header)

In [9]:
print str(header.info.header)

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##FORMAT=<ID=GT,Number=1,Type=String,Description=Genotype>
##FORMAT=<ID=AD,Number=1,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##contig=<ID=chr9>
##contig=<ID=chr8>
##contig=<ID=chr7>
##contig=<ID=chr6>
##contig=<ID=chr5>
##contig=<ID=chr4>
##contig=<ID=chr3>
##contig=<ID=chr2>
##contig=<ID=chr10>
##contig=<ID=chr1>
##contig=<ID=Y>
##contig=<ID=X>
##INFO=<ID=END,Number=1,Type=Integer,Description="Stop position of the interval">
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	SAMPLE01	SAMPLE02	SAMPLE03



In [10]:
record = None
for chromosome in chromosomes:
    print("before", chromosome)
    record = header.new_record(
        contig=chromosome,
        alleles=('ATG', 'AATGAC'),
        filter='.',
        id='.',
        start=100,
        qual=100,
        samples=[{'GT': (0, 1), 'AD': 20, 'phased': False}, {'GT': (1, 1), 'AD': 25} ,{'GT': (0, 0), 'AD': 42}]
    )
    vcf.write(record)
    print("writing", chromosome)

('before', 'chr9')
('writing', 'chr9')
('before', 'chr8')
('writing', 'chr8')
('before', 'chr7')
('writing', 'chr7')
('before', 'chr6')
('writing', 'chr6')
('before', 'chr5')
('writing', 'chr5')
('before', 'chr4')
('writing', 'chr4')
('before', 'chr3')
('writing', 'chr3')
('before', 'chr2')
('writing', 'chr2')
('before', 'chr10')
('writing', 'chr10')
('before', 'chr1')
('writing', 'chr1')
('before', 'Y')
('writing', 'Y')
('before', 'X')
('writing', 'X')


In [11]:
vcf.close()
